In [1]:
%%time
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pandas as pd
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('seaborn-dark-palette')
import ROOT
from ROOT import TLorentzVector
data = pd.DataFrame.from_csv('truth.csv')
data = data.apply(pd.to_numeric, errors='coerce')
data.drop(['i1','i2','i3'],inplace=True,axis=1)
data.drop(['X','Y','Z','T'],inplace=True,axis=1)

data['Pt'] = np.sqrt(data.Px**2 + data.Py**2)
data['Phi_x'] = np.arccos(data.Px/data.Pt)
data['Phi_y'] = np.arcsin(data.Py/data.Pt)
data['Theta'] = np.arctan(data.Pt/data.Pz)
data.head(5)

Welcome to JupyROOT 6.07/05
CPU times: user 1.17 s, sys: 758 ms, total: 1.93 s
Wall time: 3.53 s


In [2]:
%%time
pt_smear = 115e-4 # * np.mean(data.Pt)
phi_smear = 1e-4 # * np.mean(data.Phi_x)
theta_smear = 1e-4 # * np.mean(data.Theta)

data['Pt_measure'] = np.random.normal(data.Pt,pt_smear,len(data))
data['Phi_measure'] = np.sign(data.Py)*np.random.normal(data.Phi_x,phi_smear,len(data))
data['Phi_x_measure'] = np.random.normal(data.Phi_x,phi_smear,len(data))
data['Phi_y_measure'] = np.random.normal(data.Phi_y,phi_smear,len(data))
data['Theta_measure'] = np.random.normal(data.Theta,theta_smear,len(data))


#data['Px_measure'] = data.Pt_measure*np.cos(data.Phi_measure)
#data['Py_measure'] = data.Pt_measure*np.sin(data.Phi_measure)

data['Px_measure'] = data.Pt_measure*np.cos(data.Phi_x_measure)
data['Py_measure'] = data.Pt_measure*np.sin(data.Phi_y_measure)
data['Pz_measure'] = data.Pt/np.tan(data.Theta_measure)
data['E_measure'] = np.sqrt(data.Px_measure**2 + data.Py_measure**2 + data.Pz_measure**2)

CPU times: user 10.5 ms, sys: 1.37 ms, total: 11.9 ms
Wall time: 11.4 ms


In [3]:
data.head(20)

,event,ID,mother,daughter1,daughter2,Px,Py,Pz,E,Pt,...,Theta,Pt_measure,Phi_measure,Phi_x_measure,Phi_y_measure,Theta_measure,Px_measure,Py_measure,Pz_measure,E_measure
part,,,,,,,,,,,,,,,,,,,,,
0,0,300553,-1,1,2,0.1760,0.0000,4.4988,11.5028,0.176000,...,0.039102,0.165209,0.000000,0.000038,-0.000046,0.039094,0.165209,-0.000008,4.499701,4.502733
1,1,521,0,3,4,0.4054,-0.1093,2.4376,5.8299,0.419876,...,0.170576,0.436013,-0.263368,0.263193,-0.263355,0.170602,0.420999,-0.113504,2.437223,2.475920
2,2,-521,0,9,12,-0.2294,0.1093,2.0612,5.6729,0.254108,...,0.122663,0.247995,2.696880,2.696833,0.444491,0.122501,-0.223869,0.106638,2.063949,2.078792
3,3,333,1,5,6,2.0084,-1.4483,1.7238,3.1843,2.476135,...,0.962648,2.493663,-0.624957,0.624767,-0.624696,0.962602,2.022608,-1.458422,1.723971,3.031504
4,4,10321,1,7,8,-1.6030,1.3390,0.7138,2.6456,2.088667,...,1.241491,2.087113,2.445689,2.445641,0.695836,1.241731,-1.601742,1.337896,0.713241,2.205506
5,5,321,3,32,34,1.2486,-0.9178,1.0902,1.9580,1.549632,...,0.957711,1.531843,-0.633850,0.633978,-0.634054,0.957428,1.234171,-0.907488,1.090855,1.880606
6,6,-321,3,35,37,0.7598,-0.5305,0.6336,1.2263,0.926675,...,0.971070,0.913735,-0.609525,0.609494,-0.609541,0.971005,0.749206,-0.523105,0.633689,1.111985
7,7,310,4,38,39,-1.6694,0.7277,0.3581,1.9216,1.821111,...,1.376635,1.823587,2.730441,2.730501,0.411228,1.376602,-1.671655,0.728953,0.358164,1.858517
8,8,211,4,40,43,0.0664,0.6113,0.3558,0.7240,0.614896,...,1.046235,0.614256,1.462517,1.462550,1.462693,1.046129,0.066361,0.610670,0.355887,0.709914


In [ ]:
kin = data.drop(['event','ID','mother','daughter1','daughter2'],axis=1)
ax = scatter_matrix(kin, alpha=0.2, figsize=(20, 15),diagonal='kde')

In [ ]:
events = pd.DataFrame()
mom = 0
phi_mass = []
temp = TLorentzVector(0,0,0,0)
for index, row in data.iterrows():
    if row.ID == 333:
        if temp.M() != 0 and temp.M() < 1.2:
            phi_mass.append(temp.M())
        temp.SetPxPyPzE(0,0,0,0)
        mom = row['event']
    elif np.abs(row.ID) == 321 and row.mother == mom:
        temp = temp + TLorentzVector(row.Px_measure,row.Py_measure,row.Pz_measure,row.E)
        events = events.append(row)

events = events.apply(pd.to_numeric, errors='coerce')

phi_mass = np.array(phi_mass)

fig = plt.figure(figsize=(16,9))

bins = 80
ax = plt.hist(phi_mass,histtype='stepfilled',bins=bins)

In [ ]:
%%time
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from StringIO import StringIO
from ROOT import TLorentzVector
from scipy.optimize import curve_fit
import pandas as pd
import ROOT
from ROOT import TLorentzVector
data = pd.DataFrame.from_csv('truth.csv')
data = data.apply(pd.to_numeric, errors='coerce')
data.drop(['i1','i2','i3'],inplace=True,axis=1)
data.drop(['X','Y','Z','T'],inplace=True,axis=1)

data['Pt'] = np.sqrt(data.Px**2 + data.Py**2)
data['Phi_x'] = np.arccos(data.Px/data.Pt)
data['Phi_y'] = np.arcsin(data.Py/data.Pt)
data['Theta'] = np.arctan(data.Pt/data.Pz)

pt_smear = 115e-4 # * np.mean(data.Pt)
phi_smear = 1e-4 # * np.mean(data.Phi_x)
theta_smear = 1e-4 # * np.mean(data.Theta)

data['Pt_measure'] = np.random.normal(data.Pt,pt_smear*data.Pt,len(data))
data['Phi_measure'] = np.sign(data.Py)*np.random.normal(data.Phi_x,phi_smear,len(data))
data['Phi_x_measure'] = np.random.normal(data.Phi_x,phi_smear,len(data))
data['Phi_y_measure'] = np.random.normal(data.Phi_y,phi_smear,len(data))
data['Theta_measure'] = np.random.normal(data.Theta,theta_smear,len(data))

data['Px_measure'] = data.Pt_measure*np.cos(data.Phi_x_measure)
data['Py_measure'] = data.Pt_measure*np.sin(data.Phi_y_measure)
data['Pz_measure'] = data.Pt/np.tan(data.Theta_measure)
data['E_measure'] = np.sqrt(data.Px_measure**2 + data.Py_measure**2 + data.Pz_measure**2)


fig = plt.figure(num=None, figsize=(15, 15), dpi=200, facecolor='w', edgecolor='k')

num_bins = 75

def BW(Energy,Mass,Gamma):
    g = ((Mass**2.0 + Gamma**2.0)*Mass**2.0)**(1.0/2.0)
    k = (2.0 * 2.0**(1.0/2.0) * Mass * Gamma * g)/(np.pi * (Mass**(2.0)+g)**(1.0/2.0))
    return (k/((Energy**2.0-Mass**2.0)**2.0 + (Gamma*Mass)**2.0))

def BW_NonR(Energy,Mass,Gamma):
    return (((Gamma/(2.0*np.pi)))/((Energy-Mass)**2.0 + (Gamma/2.0)**2.0))

def chi_2(ys,yknown):
    total = 0
    for i in xrange(len(yknown)):
        temp = (ys[i]-yknown[i])**2.0
        if yknown[i] == 0:
            total += temp
        else :
            total += temp/yknown[i]
    return total/len(yknown)

def gauss(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

mom = 0
phi_mass = []
temp = TLorentzVector(0,0,0,0)
for index, row in data.iterrows():
    if row.ID == 333:
        if temp.M() != 0 and temp.M() < 1.2:
            phi_mass.append(temp.M())
        temp.SetPxPyPzE(0,0,0,0)
        mom = row['event']
    elif np.abs(row.ID) == 321 and row.mother == mom:
        temp = temp + TLorentzVector(row.Px_measure,row.Py_measure,row.Pz_measure,row.E)

mass_sum = np.array(phi_mass)

hist, bin_edges = np.histogram(mass_sum,bins=num_bins)
xdata = 0.5*(bin_edges[1:]+bin_edges[:-1])
ydata = hist

n, bins, patches = plt.hist(mass_sum, num_bins, histtype=u'stepfilled',facecolor='g' , alpha=0.5)

n = len(xdata)
mean = sum(xdata*ydata)/n 

x0 = np.array([mean,1])

popt, pcov = curve_fit(gauss, xdata, ydata,p0=[1,mean,1])
perr = np.sqrt(np.diag(pcov))
plt.plot(xdata,gauss(xdata,*popt),'g-', lw=4,
    label=r'$\mathrm{Gaus:\ \mu=%.3f \pm %.3f \ GeV,}\ \sigma=%.3f \pm %.3f$' %(popt[1], perr[1], popt[0], perr[0]))
plt.plot(xdata,gauss(xdata,*popt),'g-', lw=4,
    label=r'$\mathrm{Gaus: \ \chi^{2} = %.3f}$' %(chi_2(gauss(xdata,*popt),ydata)))

popt_1, pcov_1 = curve_fit(BW, xdata, ydata, p0=x0)
perr_1 = np.sqrt(np.diag(pcov_1))
plt.plot(xdata,BW(xdata,popt_1[0],popt_1[1]),'b-.', lw=4,
    label=r'$\mathrm{Relatavistic \ BW:\ Mass=%.3f \pm %.3f \ GeV,}\ \Gamma=%.3f \pm %.3f$' %(popt_1[0], perr_1[0], popt_1[1], perr_1[1]))
plt.plot(xdata,BW(xdata,popt_1[0],popt_1[1]),'b-.', lw=4,
    label=r'$\mathrm{Relatavistic \ BW: \ \chi^{2} = %.3f}$' %(chi_2(BW(xdata,popt_1[0],popt_1[1]),ydata)))

popt_2, pcov_2 = curve_fit(BW_NonR, xdata, ydata, p0=x0)
perr_2 = np.sqrt(np.diag(pcov_2))
plt.plot(xdata,BW_NonR(xdata,popt_2[0],popt_2[1]),'r--', lw=4,
    label=r'$\mathrm{Non-Rel. \ BW:\ Mass=%.3f \pm %.3f \ GeV,}\ \Gamma=%.3f \pm %.3f$' %(popt_2[0], perr_2[0], popt_2[1], perr_2[1]))
plt.plot(xdata,BW_NonR(xdata,popt_2[0],popt_2[1]),'r--', lw=4,
    label=r'$\mathrm{Non-Rel. \ BW: \ \chi^{2} = %.3f}$' %(chi_2(BW_NonR(xdata,popt_2[0],popt_2[1]),ydata)))



plt.xlabel(r'Mass (GeV)')
plt.ylabel(r'Counts (#)')
plt.legend(loc=2)
plt.show()